# Preprocess the SWC files

The raw reconstructions are read in, the soma is reduced to one point and y and z direction are switched so that z corresponds to cortical depth. The reconstruction is resampled at a fistance of 1 micron and the y-direction (so the direction that is flattened during staining) is smoothed.

In [ ]:
import sys
import pandas as pd
import datetime
import os

from neurontree import NeuronTree as nt
from neurontree.utils import get_standardized_swc

In [ ]:
root_path='D:/mpinb/data/mouse_v1'

file_path = root_path + '/raw/morph/200526/'
saving_path = root_path + '/processed/morph/nt/'

In [ ]:
meta_data_file_path = root_path + '/20200711_patchseq_metadata_mouse.csv'
cells = pd.read_csv(meta_data_file_path, sep=',', index_col=1)
cells = cells[cells['neuron_reconstruction_type'] == 'full']

In [ ]:
for file_name in list(cells.index):
    
    input_file = file_path + str(file_name) + '_transformed.swc'
    print('Preprocessing file %s ...'%input_file)
    swc = pd.read_csv(input_file, delim_whitespace=True, comment='#',
                      names=['n', 'type', 'x', 'y', 'z', 'radius', 'parent'], index_col=False, header=None)
    # switch y and z since y corresponds to cortical depth
    swc = swc.rename(columns={'y': 'z', 'z': 'y'})
    print(swc.loc[0,"z"])
    # soma center for standardization
    rotated_swc = get_standardized_swc(swc, pca_rot=False)


    N = nt.NeuronTree(swc=rotated_swc)
    # Resample neuron at distance 1 micron
    N = N.resample_tree(dist=1)
    # Smooth neurites in y direction
    N = N.smooth_neurites(dim=1, window_size=21)

    N.write_to_swc(file_name=str(file_name), path=saving_path)

In [ ]:
swc = pd.read_csv(input_file, delim_whitespace=True, comment='#',
                      names=['n', 'type', 'x', 'y', 'z', 'radius', 'parent'], index_col=False, header=None)